In [1]:
import numpy as np
import pandas as pd
import rasterio
from affine import Affine
import matplotlib.pyplot as plt
import matplotlib.image as mping
import matplotlib.colors as colors

### https://automating-gis-processes.github.io/CSC18/index.html
### https://rasterio.readthedocs.io/en/latest/quickstart.html
### http://darribas.org/gds15/content/labs/lab_03.html

### Each monthly tarball contains 2 files, the first one is the average DNB radiance, the second contains the number of cloud-free observations used in the average.

## The "vcmsl" version, that includes the stray-light corrected data, will have more data coverage toward the poles, but will be of reduced quality.

In [2]:
file_in = 'SVDNB/SVDNB_npp_20191201-20191231_75N060E_vcmcfg_v10_c202001140900/SVDNB_npp_20191201-20191231_75N060E_vcmcfg_v10_c202001140900.avg_rade9h.tif'

In [3]:
dataset = rasterio.open(file_in)
dataset.count

1

In [4]:
dataset.bounds

BoundingBox(left=59.99791666665, bottom=0.0020827333499937595, right=179.99791762665, top=75.00208333335)

In [5]:
for k in dataset.meta:
    print(k,dataset.meta[k])

driver GTiff
dtype float32
nodata None
width 28800
height 18000
count 1
crs EPSG:4326
transform | 0.00, 0.00, 60.00|
| 0.00,-0.00, 75.00|
| 0.00, 0.00, 1.00|


## the map covers from the left -180 to the right -60 and from bottom 0 to top 75
### I Think they are latitude and longitude

In [6]:
dataset.height, dataset.width

(18000, 28800)

In [7]:
dataset.crs

CRS.from_epsg(4326)

## Corordinate refernce system 
https://epsg.io/ 
####  4326 is latitude , longitude..Good!

In [8]:
band1 = dataset.read(1)
band1.shape

(18000, 28800)

In [9]:
band1[0:10,0:10]

array([[0.85, 0.84, 0.86, 0.86, 0.89, 0.91, 0.9 , 0.85, 0.85, 0.84],
       [0.87, 0.87, 0.86, 0.85, 0.87, 0.89, 0.88, 0.87, 0.87, 0.84],
       [0.84, 0.85, 0.85, 0.87, 0.9 , 0.88, 0.94, 0.93, 0.9 , 0.87],
       [0.85, 0.84, 0.83, 0.86, 0.9 , 0.96, 0.96, 0.93, 0.91, 0.88],
       [0.85, 0.86, 0.9 , 0.94, 0.97, 1.02, 0.96, 0.97, 0.91, 0.87],
       [0.89, 0.88, 0.92, 0.97, 0.96, 0.99, 0.96, 0.94, 0.91, 0.88],
       [0.89, 0.87, 0.93, 0.91, 0.95, 0.94, 0.91, 0.93, 0.92, 0.89],
       [0.91, 0.9 , 0.92, 0.95, 0.96, 0.93, 0.93, 0.9 , 0.87, 0.9 ],
       [0.89, 0.92, 0.91, 0.91, 0.93, 0.94, 0.91, 0.89, 0.89, 0.85],
       [0.9 , 0.91, 0.91, 0.9 , 0.94, 0.94, 0.96, 0.87, 0.87, 0.85]],
      dtype=float32)

### A dataset’s transform is an affine transformation matrix that maps pixel locations in (row, col) coordinates to (x, y) 

In [10]:
dataset.transform

Affine(0.0041666667, 0.0, 59.99791666665,
       0.0, -0.0041666667, 75.00208333335)

In [11]:
dataset.transform.a, dataset.transform.e, dataset.transform.c, dataset.transform.f

(0.0041666667, -0.0041666667, 59.99791666665, 75.00208333335)

# Convert to Corodinate

In [12]:
dataset.transform * (0, 0) ,dataset.transform * (dataset.width, dataset.height)

((59.99791666665, 75.00208333335), (179.99791762665, 0.0020827333499937595))

In [13]:
dataset.transform * (0,9) , dataset.transform * (0,dataset.height) ,dataset.transform * (dataset.width, 0) 

((59.99791666665, 74.96458333305),
 (59.99791666665, 0.0020827333499937595),
 (179.99791762665, 75.00208333335))

In [14]:
(dataset.transform * (0, 0))[0]

59.99791666665

### each row corrsponds to a longitude
### each column corresponds to a latitude


In [26]:
column_list = []
longitude_list = []
latitude_list= []
for i in range(dataset.height):
    l1 =(dataset.transform * (0,i))[1]

    if l1<=28 and l1>= 8:
        column_list.append(i)
        latitude_list.append(l1)
row_list = []      
for j in range(dataset.height):
    l2 =(dataset.transform * (j,0))[0]
    if l2 <=88 and l2 >=76:
        row_list.append(j)
        longitude_list.append(l2)
                
longitude_list = np.array(longitude_list)
latitude_list = np.array(latitude_list)
print(len(column_list),len(row_list), len(longitude_list),len(latitude_list))

4800 2880 2880 4800


In [27]:
row_list =  row_list[1000:1200]
column_list = column_list[100:140]

longitude_list=  longitude_list[1000:1200]
latitude_list = latitude_list[100:140]

In [28]:
light = band1[row_list,:][:,column_list]
df = np.transpose([np.repeat(longitude_list, len(latitude_list)), 
              np.tile(latitude_list, len(longitude_list)),light.flatten()])

In [29]:
df

array([[80.16875016, 27.58124962,  0.25      ],
       [80.16875016, 27.57708295,  0.27000001],
       [80.16875016, 27.57291629,  0.30000001],
       ...,
       [80.99791683, 27.42708295,  0.28      ],
       [80.99791683, 27.42291629,  0.28      ],
       [80.99791683, 27.41874962,  0.30000001]])

In [30]:
df = pd.DataFrame(df)
df.columns = ['longitude','latitude','NB']
df.to_pickle("data.pickle")

### multiply with transform to know coordinate

In [21]:
dataset.index(0,0)

(18000, 43200)

In [10]:
x, y = (dataset.bounds.right , dataset.bounds.top )
row, col = dataset.index(x, y)
print(x,y)
print(row,col)
band1[row-1, col-1]

-60.00208237334998 75.00208333335
0 28800


0.04

In [41]:
for k in dataset.meta:
    print(k,dataset.meta[k])

driver GTiff
dtype float32
nodata None
width 28800
height 18000
count 1
crs EPSG:4326
transform | 0.00, 0.00, 60.00|
| 0.00,-0.00, 75.00|
| 0.00, 0.00, 1.00|


In [ ]:
if isinstance(dataset.transform, Affine):
     transform = dataset.transform
else:
     transform = dataset.affine

N = dataset.width
M = dataset.height
dx = transform.a
dy = transform.e
minx = transform.c
maxy = transform.f

# Read the image data, flip upside down if necessary
data_in = dataset.read(1)
if dy < 0:
  dy = -dy
  data_in = np.flip(data_in, 0)

print('Data minimum, maximum = ', np.amin(data_in), np.amax(data_in))

# Generate X and Y grid locations
xdata = minx + dx/2 + dx*np.arange(N)
ydata = maxy - dy/2 - dy*np.arange(M-1,-1,-1)

# Scale the velocities by the log of the data.
d = np.log(np.clip(data_in, 1, 3000))
data_scale = (255*(d - np.amin(d))/np.ptp(d)).astype(np.uint8)

# Construct an RGB table using a log scale between 1 and 3000 m/year.
vel = np.exp(np.linspace(np.log(1), np.log(3000), num=256))
hue = np.arange(256)/255.0
sat = np.clip(1./3 + vel/187.5, 0, 1)
value = np.zeros(256) + 0.75
hsv = np.stack((hue, sat, value), axis=1)
rgb = colors.hsv_to_rgb(hsv)
# Be sure the first color (the background) is white
rgb[0,:] = 1
cmap = colors.ListedColormap(rgb, name='velocity')

extent = [xdata[0], xdata[-1], ydata[0], ydata[-1]]
plt.figure(figsize=(8,8))
fig = plt.imshow(data_scale, extent=extent, origin='lower', cmap=cmap)
plt.title(file_in)
# Hide the axes and remove the space around them
plt.axis('off')
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
tickval = [1, 10, 100, 1000, 3000]
t = np.log(tickval)
cb = plt.colorbar(fig, ticks=255*(t - t[0])/(t[-1] - t[0]), shrink=0.5)
cb.set_label('Velocity Magnitude (m/year)')
cb.ax.set_yticklabels(tickval)
plt.savefig("python_sample.png", dpi=300, bbox_inches='tight', pad_inches=0.5)
plt.show()

Data minimum, maximum =  -1.02 21473.26


In [11]:
file_in = 'SVDNB/SVDNB_npp_20191201-20191231_75N180W_vcmcfg_v10_c202001140900.cf_cvg.tif'

In [15]:
dataset = rasterio.open(file_in)
for k in dataset.meta:
    print(k,dataset.meta[k])
print("Bounds {}".format(dataset.bounds))

driver GTiff
dtype uint16
nodata None
width 28800
height 18000
count 1
crs EPSG:4326
transform | 0.00, 0.00,-180.00|
| 0.00,-0.00, 75.00|
| 0.00, 0.00, 1.00|
Bounds BoundingBox(left=-180.00208333335, bottom=0.0020827333499937595, right=-60.00208237334998, top=75.00208333335)


In [17]:
band1 = dataset.read(1)
print(band1.shape)
band1[0:10,0:10]

(18000, 28800)


array([[42, 41, 41, 42, 43, 43, 43, 43, 42, 42],
       [42, 41, 42, 42, 43, 43, 43, 43, 42, 42],
       [41, 41, 42, 42, 41, 40, 41, 41, 41, 42],
       [41, 41, 41, 41, 40, 40, 40, 40, 40, 40],
       [41, 41, 40, 40, 40, 40, 40, 40, 40, 40],
       [40, 40, 40, 40, 41, 41, 41, 41, 41, 41],
       [41, 41, 41, 41, 41, 41, 41, 41, 41, 40],
       [41, 41, 41, 41, 41, 41, 41, 41, 41, 40],
       [41, 41, 41, 41, 41, 41, 41, 41, 41, 40],
       [40, 40, 41, 41, 41, 41, 41, 41, 41, 41]], dtype=uint16)

### geopandas:  https://geopandas.org/gallery/index.html